In [17]:
import pandas as pd
import json
from pandas import json_normalize
import numpy as np

# 최대 출력 컬럼 제한 해제
pd.set_option('display.max_colwidth', None)

##### 상관 계수를 이용하여 대체기물 추천

In [18]:
units_df = pd.read_csv('유닛 조합과 별갯수 테이블.csv', encoding='utf-8')

units_df.head(1)

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,시비르,리븐,...,코부코,다리우스,아펠리오스,쓰레쉬,초가스,말파이트,가렌,티모,자야,Combination
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,2,2,...,0,0,0,0,0,0,0,0,0,"TFT11_Sivir, TFT11_Riven, TFT11_Shen, TFT11_TahmKench, TFT11_Galio, TFT11_Kaisa, TFT11_Sylas, TFT11_Udyr, TFT11_Kayle"


In [19]:
units_df = units_df.drop('케일', axis=1)

only_units = units_df.loc[:, '시비르':'자야']

# `시비르`부터 `자야`까지의 열을 선택하여 모든 0이 아닌 값을 1로 바꿉니다.
only_units.loc[:, '시비르':'자야'] = only_units.loc[:, '시비르':'자야'].applymap(lambda x: 1 if x != 0 else x)

only_units.head()

,시비르,리븐,쉔,탐 켄치,갈리오,카이사,사일러스,우디르,케이틀린,렉사이,...,신드라,코부코,다리우스,아펠리오스,쓰레쉬,초가스,말파이트,가렌,티모,자야
0,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
import pandas as pd

# 주어진 데이터프레임에서 상관관계 행렬 계산
correlation_matrix = only_units.corr()

# 빈 데이터프레임 생성
max_corr_df = pd.DataFrame(columns=['챔피언', '대체기물_유닛'])

# 각 열에 대해 가장 높은 상관관계를 가진 열을 찾아서 데이터프레임에 추가
for column in correlation_matrix.columns:
    # 자기 자신은 제외
    corr_values = correlation_matrix[column].drop(column)
    max_corr_column = corr_values.idxmax()
    max_corr_value = corr_values.max()
    
    # 데이터프레임에 추가
    max_corr_df = pd.concat([max_corr_df, pd.DataFrame({'챔피언': [column], '대체기물_유닛': [max_corr_column]})], ignore_index=True)

max_corr_df

,챔피언,대체기물_유닛
0,시비르,바드
1,리븐,갈리오
2,쉔,일라오이
3,탐 켄치,바드
4,갈리오,리븐
5,카이사,티모
6,사일러스,알룬
7,우디르,오른
8,케이틀린,세나
9,렉사이,나르


##### 비슷한 아이템이 삼신기인 유닛을 대체기물로 추천

In [21]:
item_df = pd.read_csv('유닛별 통계 + 아이템 추천(태블로용 최종).csv', encoding='utf-8')

item_df.head()

,평균 순위,픽률,챔피언,추천 아이템1,추천 아이템2,추천 아이템3,대체 아이템1,대체 아이템2,대체기물_유닛기준,대체기물_아이템기준,대체기물_유닛기준.1,대체기물_아이템기준.1
0,4.746781,5.017227,가렌,피바라기,구원,거인의 결의,가고일 돌갑옷,워모그의 갑옷,자이라,우디르,자이라,우디르
1,4.623248,26.119724,갈리오,가고일 돌갑옷,가고일 돌갑옷,워모그의 갑옷,구원,용의 발톱,리븐,아무무,리븐,아무무
2,4.643192,4.586563,나르,피바라기,거인의 결의,거인의 결의,정의의 손길,스테락의 도전,렉사이,세트,렉사이,세트
3,4.333333,15.665375,노틸러스,가고일 돌갑옷,가고일 돌갑옷,워모그의 갑옷,구원,굳건한 심장,릴리아,오른,릴리아,오른
4,4.409976,17.700258,니코,가고일 돌갑옷,태양불꽃 망토,워모그의 갑옷,구원,굳건한 심장,탐 켄치,노틸러스,탐 켄치,노틸러스


In [22]:
import pandas as pd
from scipy.spatial.distance import euclidean

# 아이템 컬럼만 선택하여 유사도 계산에 사용할 데이터 프레임 생성
item_data = item_df[['추천 아이템1', '추천 아이템2', '추천 아이템3', '대체 아이템1', '대체 아이템2']]

# 아이템 이름을 숫자로 변환하는 매핑 딕셔너리 생성
item_mapping = {}
unique_items = set(item_df[['추천 아이템1', '추천 아이템2', '추천 아이템3', '대체 아이템1', '대체 아이템2']].values.flatten())
for i, item in enumerate(unique_items):
    item_mapping[item] = i

# 아이템을 숫자로 변환
item_data_numeric = item_data.replace(item_mapping)

# 데이터프레임 초기화
similarities_df = pd.DataFrame(columns=['챔피언', '대체기물_아이템'])

# 모든 챔피언에 대해 가장 높은 아이템 유사도를 가진 챔피언 찾기
for champion in item_df['챔피언']:
    max_similarity = float('inf')
    most_similar_champion = ""
    champion_row = item_df[item_df['챔피언'] == champion].index[0]
    
    for other_champion in item_df['챔피언']:
        if other_champion != champion:
            other_champion_row = item_df[item_df['챔피언'] == other_champion].index[0]
            dist = euclidean(item_data_numeric.iloc[champion_row], item_data_numeric.iloc[other_champion_row])
            if dist < max_similarity:
                max_similarity = dist
                most_similar_champion = other_champion
    
    similarities_df = pd.concat([similarities_df, pd.DataFrame({'챔피언': [champion], '대체기물_아이템': [most_similar_champion]})], ignore_index=True)

similarities_df

,챔피언,대체기물_아이템
0,가렌,우디르
1,갈리오,아무무
2,나르,세트
3,노틸러스,오른
4,니코,노틸러스
5,다리우스,나르
6,다이애나,말파이트
7,라칸,초가스
8,럭스,잔나
9,렉사이,요릭


##### 두 테이블, 챔피언 명을 기준으로 join

In [23]:
# max_corr_df와 similarities_df를 챔피언 컬럼을 기준으로 조인
대체_기물_테이블 = pd.merge(max_corr_df, similarities_df, on='챔피언')

대체_기물_테이블

,챔피언,대체기물_유닛,대체기물_아이템
0,시비르,바드,세나
1,리븐,갈리오,요네
2,쉔,일라오이,노틸러스
3,탐 켄치,바드,노틸러스
4,갈리오,리븐,아무무
5,카이사,티모,케이틀린
6,사일러스,알룬,볼리베어
7,우디르,오른,볼리베어
8,케이틀린,세나,카이사
9,렉사이,나르,요릭


##### 기존 유닛 통계 + 아이템 추천 최종 테이블에 대체기물 컬럼 추가 (join)

In [24]:
final_units_df = pd.read_csv('유닛별 통계 + 아이템 추천(태블로용 최종).csv', encoding='utf-8')

final_units_df.head()

,평균 순위,픽률,챔피언,추천 아이템1,추천 아이템2,추천 아이템3,대체 아이템1,대체 아이템2,대체기물_유닛기준,대체기물_아이템기준,대체기물_유닛기준.1,대체기물_아이템기준.1
0,4.746781,5.017227,가렌,피바라기,구원,거인의 결의,가고일 돌갑옷,워모그의 갑옷,자이라,우디르,자이라,우디르
1,4.623248,26.119724,갈리오,가고일 돌갑옷,가고일 돌갑옷,워모그의 갑옷,구원,용의 발톱,리븐,아무무,리븐,아무무
2,4.643192,4.586563,나르,피바라기,거인의 결의,거인의 결의,정의의 손길,스테락의 도전,렉사이,세트,렉사이,세트
3,4.333333,15.665375,노틸러스,가고일 돌갑옷,가고일 돌갑옷,워모그의 갑옷,구원,굳건한 심장,릴리아,오른,릴리아,오른
4,4.409976,17.700258,니코,가고일 돌갑옷,태양불꽃 망토,워모그의 갑옷,구원,굳건한 심장,탐 켄치,노틸러스,탐 켄치,노틸러스


In [25]:
final_units_df = pd.merge(final_units_df, 대체_기물_테이블, on='챔피언')

final_units_df.rename(columns={'아이템1': '추천 아이템1'}, inplace=True)
final_units_df.rename(columns={'아이템2': '추천 아이템2'}, inplace=True)
final_units_df.rename(columns={'아이템3': '추천 아이템3'}, inplace=True)

final_units_df.rename(columns={'아이템4': '대체 아이템1'}, inplace=True)
final_units_df.rename(columns={'아이템5': '대체 아이템2'}, inplace=True)

final_units_df.rename(columns={'대체기물_유닛': '대체기물_유닛기준'}, inplace=True)
final_units_df.rename(columns={'대체기물_아이템': '대체기물_아이템기준'}, inplace=True)

# 원하는 컬럼 드랍하기
final_units_df.drop(columns=['대체기물_유닛기준.1', '대체기물_아이템기준.1'], inplace=True)

# 중복된 컬럼을 삭제하여 하나만 유지
final_units_df = final_units_df.loc[:, ~final_units_df.columns.duplicated()]

final_units_df

,평균 순위,픽률,챔피언,추천 아이템1,추천 아이템2,추천 아이템3,대체 아이템1,대체 아이템2,대체기물_유닛기준,대체기물_아이템기준
0,4.746781,5.017227,가렌,피바라기,구원,거인의 결의,가고일 돌갑옷,워모그의 갑옷,자이라,우디르
1,4.623248,26.119724,갈리오,가고일 돌갑옷,가고일 돌갑옷,워모그의 갑옷,구원,용의 발톱,리븐,아무무
2,4.643192,4.586563,나르,피바라기,거인의 결의,거인의 결의,정의의 손길,스테락의 도전,렉사이,세트
3,4.333333,15.665375,노틸러스,가고일 돌갑옷,가고일 돌갑옷,워모그의 갑옷,구원,굳건한 심장,릴리아,오른
4,4.409976,17.700258,니코,가고일 돌갑옷,태양불꽃 망토,워모그의 갑옷,구원,굳건한 심장,탐 켄치,노틸러스
5,4.553977,7.579673,다리우스,피바라기,태양불꽃 망토,거인의 결의,이온 충격기,저녁갑주,야스오,카직스
6,4.429733,33.096469,다이애나,가고일 돌갑옷,구원,워모그의 갑옷,태양불꽃 망토,굳건한 심장,리 신,말파이트
7,3.907106,31.061585,라칸,이온 충격기,가고일 돌갑옷,워모그의 갑옷,모렐로노미콘,구원,오공,카이사
8,4.771670,5.092593,럭스,보석 건틀릿,보석 건틀릿,쇼진의 창,스태틱의 단검,내셔의 이빨,아무무,잔나
9,4.690307,4.554264,렉사이,수호자의 맹세,구원,구원,태양불꽃 망토,워모그의 갑옷,나르,가렌


In [26]:
final_units_df.to_csv('유닛별 통계 + 아이템 추천 + 대체 기물 추천(태블로용 최종).csv', index=False)